In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
%matplotlib inline

In [ ]:
def spline_maker(xvals,fvals,qvals):
    # m = fvals.size
    # note, from above, n = m-1
    
    n = fvals.size - 1
    df = fvals[1:]-fvals[0:n]
    dx = xvals[1:]-xvals[0:n]
    dfdx = df/dx
    svals = np.zeros(ivals.size)
    
    rhs = dfdx[1:] - dfdx[0:n-1]
    diag = 2./3.*(dx[1:] + dx[0:n-1])
    data = np.array([diag,dx[1:]/3.,dx[0:n-1]/3.])
    dvals = np.array([0,-1,1])
    Amat = spdiags(data, dvals, n-1, n-1, format='csc')
    bvec = spsolve(Amat,rhs)
    
    bvec = np.append(0.,bvec)
    
    cvec = dfdx - 2./3.*dx*bvec - dx/3.*np.append(bvec[1:],0.)
    avec = (dfdx - dx*bvec - cvec)/(dx**2.)
    
    for jj in range(1,n+1):
        
        indsr = qvals < xvals[jj] 
        indsl = qvals >= xvals[jj-1]
        inds = indsl*indsr
        
        dxloc = qvals[inds] - xvals[jj-1]
        svals[inds] = avec[jj-1]*dxloc**3. + bvec[jj-1]*dxloc**2. + cvec[jj-1]*dxloc + fvals[jj-1]
        
    return svals

## Cubic Splines

Again, we start our discussion with a data set 

$$
\left\{x_{j},f_{j}\right\}_{j=0}^{n}.
$$

As discussed in class, we now define our _cubic splines_ $S_{j}(x)$ to be third order polynomials, i.e. 

$$
S_{j}(x) = a_{j}(x-x_{j})^{3} + b_{j}(x-x_{j})^{2} + c_{j}(x-x_{j}) + d_{j},
$$

such that 

\begin{align}
S_{j}(x_{j}) = & f_{j}, ~ j=0,\cdots,n-1\\
S_{j}(x_{j+1}) = & S_{j+1}(x_{j+1}), ~ j=0,\cdots,n-2 \\
S'_{j}(x_{j+1}) = & S'_{j+1}(x_{j+1}), ~ j=0,\cdots,n-2 \\
S''_{j}(x_{j+1}) = & S''_{j+1}(x_{j+1}), ~ j=0,\cdots,n-2 
\end{align}

which is to say, we require that we interpolate the data, and the each spline as as its first and second derivatives be continuous at each node.  Finally, we require that 

$$
S_{n-1}(x_{n}) = f_{n}, ~ S''_{0}(x_{0})=0, ~ S''_{n-1}(x_{n}) = 0.
$$

The second derivative requirements at the boundary of the data make these _natural splines_.  

We readily see then that $d_{j}= f_{j}$.  Define 

$$
\delta x_{j} = x_{j+1} -x_{j}, ~ \delta f_{j} = f_{j+1} - f_{j}.
$$

Then from above we get the system of equations for  $j=0,\cdots,n-2$

\begin{align}
a_{j}(\delta x_{j})^{2} + b_{j}\delta x_{j} + c_{j} = & \frac{\delta f_{j}}{\delta x_{j}},\\
3a_{j}(\delta x_{j})^{2} + 2b_{j}\delta x_{j} + c_{j} = & c_{j+1},\\
3a_{j}\delta x_{j} + b_{j} = & b_{j+1} 
\end{align}

The end point conditions give us

$$
a_{n-1}\left(\delta x_{n-1}\right)^{2} + b_{n-1}\delta x_{n-1} + c_{n-1} = \frac{\delta f_{n-1}}{\delta x_{n-1}},
$$

and

$$
b_{0}=0, ~ 3a_{n-1}\delta x_{n-1} + b_{n-1} = 0.
$$


**Problem 1**: Show that 

$$
a_{j} = \frac{b_{j+1}-b_{j}}{3\delta x_{j}}, ~ j=0, \cdots, n-2
$$

and

$$
a_{n-1} = -\frac{b_{n-1}}{3\delta x_{n-1}}.
$$

**Problem 2**: Using the result from Problem 1 and, for $j=0,\cdots,n-2$, the remaining two equations 
\begin{align*}
a_{j}(\delta x_{j})^{2} + b_{j}\delta x_{j} + c_{j} = & \frac{\delta f_{j}}{\delta x_{j}},\\
3a_{j}(\delta x_{j})^{2} + 2b_{j}\delta x_{j} + c_{j} = & c_{j+1}
\end{align*}

show that for $j=0,\cdots,n-3$ that 

$$
\frac{1}{3}\delta x_{j} b_{j} + \frac{2}{3}\left(\delta x_{j} + \delta x_{j+1}\right)b_{j+1} + \frac{1}{3}\delta x_{j+1}b_{j+2} = \frac{\delta f_{j+1}}{\delta x_{j+1}} - \frac{\delta f_{j}}{\delta x_{j}}
$$

and for $j=n-2$ we have 

$$
\frac{1}{3}\delta x_{n-2} b_{n-2} + \frac{2}{3}\left(\delta x_{n-2} + \delta x_{n-1}\right)b_{n-1} = \frac{\delta f_{n-1}}{\delta x_{n-1}} - \frac{\delta f_{n-2}}{\delta x_{n-2}}
$$


**Problem 3**: Remembering that $b_{0}=0$, how would you write the result from Problem 2 as a matrix/vector system of the form

$$
A{\bf b} = {\bf f}
$$

where

$$
{\bf b} = \begin{pmatrix} b_{1} \\ b_{2} \\ \vdots \\ b_{n-1}\end{pmatrix}, ~ {\bf f} = \begin{pmatrix} \frac{\delta f_{1}}{\delta x_{1}} - \frac{\delta f_{0}}{\delta x_{0}}\\ \frac{\delta f_{2}}{\delta x_{2}} - \frac{\delta f_{1}}{\delta x_{1}} \\ \vdots \\ \frac{\delta f_{n-1}}{\delta x_{n-1}} - \frac{\delta f_{n-2}}{\delta x_{n-2}}\end{pmatrix}
$$

Note, the answer is written down in the Lecture Ten notes, but try to figure it out first without looking it up.

**Problem 4**: Using our favorite function

$$
f(x) = \frac{1}{1+x^{2}}, ~ -1\leq x \leq 1,
$$

we can now test our spline approximation scheme (see the code above).  Develop a couple of numerical experiments to determine how well our cubic spline approximation performs (again, you can peak at Lecture Ten for ideas here, but try it first before looking it up). 